In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import dask.array as da
import dask
import h5py

import logging
import sys
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

/home/wassname/.pyenv/versions/3.5.3/envs/jupyter3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from world_models_sonic.models.vae import VAE6, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.helpers.dataset import load_cache_data
from world_models_sonic.models.rnn import MDNRNN2
from world_models_sonic.models.inverse_model import InverseModel
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic.custom_envs.env import make_env
from world_models_sonic import config
from world_models_sonic.helpers.deep_rl import PPOAgent, run_iterations, SonicWorldModelDeepRL

Importing 0 potential games...
Imported 0 games


# Init

In [5]:
cuda = torch.cuda.is_available()
env_name = 'sonic256'
z_dim = 256  # latent dimensions

# RNN
action_dim = 10
image_size = 256

verbose = False  # Set this true to render (and make it go slower)

# NAME ='RNN_v3b_256im_512z_1512_v5_greenfield'
NAME = 'RNN_v3b_256im_512z_v6_greenfield'
ppo_save_file = './outputs/models/PPO_greenfields_256z_v5_wd_in_net.pkl'


save_file_rnn = './outputs/{NAME}/mdnrnn_state_dict.pkl'.format(NAME=NAME)
save_file_vae = './outputs/{NAME}/vae_state_dict.pkl'.format(NAME=NAME)
save_file_finv = './outputs/{NAME}/finv_state_dict.pkl'.format(NAME=NAME)

if not os.path.isdir('./outputs/{NAME}'.format(NAME=NAME)):
    os.makedirs('./outputs/{NAME}'.format(NAME=NAME))

# Log to file and stream
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(NAME)

# Load VAE

In [6]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE6(image_size=image_size, z_dim=32, conv_dim=48, code_dim=8, k_dim=z_dim)
if cuda:
    vae.cuda()
    
# # Resume    
if os.path.isfile(save_file_vae):
    state_dict = torch.load(save_file_vae)
    vae.load_state_dict(state_dict)
    print('loaded save_file {save_file}'.format(save_file=save_file_vae))

loaded save_file ./outputs/RNN_v3b_256im_512z_v6_greenfield/vae_state_dict.pkl


# Load RNN

In [7]:
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, 128, 3, 0.0


mdnrnn = MDNRNN2(z_dim, action_dim, hidden_size, n_mixture, temp)

if cuda:
    mdnrnn = mdnrnn.cuda()
    
# # Resume
if os.path.isfile(save_file_rnn):
    state_dict = torch.load(save_file_rnn)
    mdnrnn.load_state_dict(state_dict)
    print('loaded {save_file}'.format(save_file=save_file_rnn))

loaded ./outputs/RNN_v3b_256im_512z_v6_greenfield/mdnrnn_state_dict.pkl


# FInverse Model

In [8]:
finv = InverseModel(z_dim, action_dim, hidden_size=256).cuda()

# Resume
if os.path.isfile(save_file_finv):
    state_dict = torch.load(save_file_finv)
    finv.load_state_dict(state_dict)
    print('loaded {save_file}'.format(save_file=save_file_finv))

loaded ./outputs/RNN_v3b_256im_512z_v6_greenfield/finv_state_dict.pkl


# Init

In [9]:
world_model = WorldModel(vae, mdnrnn, finv)
world_model = world_model.eval() # Samples without randomness
if cuda:
    world_model = world_model.cuda()

# summarize

In [10]:
from IPython.display import display

with torch.no_grad():
    img = np.random.randn(image_size, image_size, 3)
    action = np.array(np.random.randint(0,action_dim))[np.newaxis]
    action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
    gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
    if cuda:
        gpu_img = gpu_img.cuda()
    with TorchSummarizeDf(vae) as tdf:
        x, mu_vae, logvar_vae = vae.forward(gpu_img)
        z = vae.sample(mu_vae, logvar_vae)
        df_vae = tdf.make_df()

    display(df_vae[df_vae.level<2])
    
    with TorchSummarizeDf(mdnrnn) as tdf: 
        pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)))
        z_next = mdnrnn.sample(pi, mu, sigma)
        df_mdnrnn = tdf.make_df()
    
    display(df_mdnrnn)
    

    with TorchSummarizeDf(finv) as tdf:
        finv(z.repeat((1,2,1)), z_next)   
        df_finv = tdf.make_df()
    display(df_finv)

    with TorchSummarizeDf(world_model) as tdf:
        world_model(gpu_img, action)
        df_world_model = tdf.make_df()
    display(df_world_model[df_world_model.level<2])
    
    del img, action, gpu_img, x, mu, z, z_next, mu_vae, pi, sigma, logvar_vae

Total parameters 8909862
Total trainable parameters 8909862


,name,class_name,input_shape,output_shape,nb_params,level
4,encoder.0,BasicConv2d,"[(-1, 3, 256, 256)]","[(-1, 48, 256, 256)]",1440,1
42,encoder.1,ConvBlock5,"[(-1, 48, 256, 256)]","[(-1, 96, 128, 128)]",93213,1
80,encoder.2,ConvBlock5,"[(-1, 96, 128, 128)]","[(-1, 144, 64, 64)]",281034,1
118,encoder.3,ConvBlock5,"[(-1, 144, 64, 64)]","[(-1, 192, 32, 32)]",566055,1
156,encoder.4,ConvBlock5,"[(-1, 192, 32, 32)]","[(-1, 240, 16, 16)]",948276,1
194,encoder.5,ConvBlock5,"[(-1, 240, 16, 16)]","[(-1, 288, 8, 8)]",1427697,1
232,encoder.6,ConvBlock5,"[(-1, 288, 8, 8)]","[(-1, 32, 8, 8)]",351550,1
233,mu,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,0
234,logvar,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,0
235,z,Linear,"[(-1, 256)]","[(-1, 2048)]",526336,0


Total parameters 1778688
Total trainable parameters 1778688


,name,class_name,input_shape,output_shape,nb_params,level
1,rnn,LSTM,"[[(-1, 2, 266)], [[(-1, 1, 128)], [(-1, 1, 128...","[[(-1, 2, 128)], [[(-1, 1, 128)], [(-1, 1, 128...",202752,0
2,ln1,Linear,"[(-1, 128), (-1, 128)]","[(-1, 128), (-1, 128)]",16512,0
3,ln2,Linear,"[(-1, 128), (-1, 128)]","[(-1, 640), (-1, 640)]",82560,0
4,mdn,Linear,"[(-1, 640), (-1, 640)]","[(-1, 2304), (-1, 2304)]",1476864,0


Total parameters 199690
Total trainable parameters 199690


,name,class_name,input_shape,output_shape,nb_params,level
1,ln1,Linear,"[(-1, 2, 512)]","[(-1, 2, 256)]",131328,0
2,ln2,Linear,"[(-1, 2, 256)]","[(-1, 2, 256)]",65792,0
3,ln3,Linear,"[(-1, 2, 256)]","[(-1, 2, 10)]",2570,0


Total parameters 10888240
Total trainable parameters 10888240


,name,class_name,input_shape,output_shape,nb_params,level
233,vae.mu,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,1
234,vae.logvar,Linear,"[(-1, 2048)]","[(-1, 256)]",524544,1
235,vae.z,Linear,"[(-1, 256)]","[(-1, 2048)]",526336,1
465,vae.sigmoid,Sigmoid,"[(-1, 3, 256, 256)]","[(-1, 3, 256, 256)]",0,1
466,mdnrnn.rnn,LSTM,"[[(-1, 1, 266)], [[(-1, 1, 128)], [(-1, 1, 128...","[[(-1, 1, 128)], [[(-1, 1, 128)], [(-1, 1, 128...",202752,1
467,mdnrnn.ln1,Linear,"[(-1, 128)]","[(-1, 128)]",16512,1
468,mdnrnn.ln2,Linear,"[(-1, 128)]","[(-1, 640)]",82560,1
469,mdnrnn.mdn,Linear,"[(-1, 640)]","[(-1, 2304)]",1476864,1


In [11]:
%debug

2018-05-25 18:14:05,565 - root - ERROR: No traceback has been produced, nothing to debug.


# Env wrappers

In [12]:
from deep_rl.utils import Config
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody

from deep_rl.component.task import ParallelizedTask

# Train

In [19]:
# A special world model network
# I can't put it as a wrapper since then pytorch wouldn't multithread
from deep_rl.network.network_utils import BaseNet
from deep_rl.network.network_heads import ActorCriticNet

import skimage.color
import cv2

class CategoricalActorCriticNet(nn.Module, BaseNet):
    def __init__(self,
                 state_dim,
                 action_dim,
                 phi_body=None,
                 actor_body=None,
                 critic_body=None,
                 world_model_fn=None,
                 gpu=-1,
                 render=False
                ):
        super(CategoricalActorCriticNet, self).__init__()
        self.world_model = world_model_fn()
        self.z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size
        self.network = ActorCriticNet(self.z_state_dim, action_dim, phi_body, actor_body, critic_body)
        
        self.viewer=None
        self.max_hidden_states = 6
        self.hidden_state = None
        self.set_gpu(gpu)

    def predict(self, obs, action=None):
        self.img = obs
        with torch.no_grad():
            # Input is (batch, samples...)
            obs = self.tensor(obs).transpose(1, 3)

            # Use world model to transform obs
            if self.hidden_state and (obs.size(0)!=self.hidden_state[0].size(0) or obs.size(1)!=self.hidden_state[0].size(1)):
                self.hidden_state=None
            z_next, z, hidden_state = self.world_model.forward(obs, hidden_state=self.hidden_state)

            hidden_state = [h.data for h in hidden_state]  # Otherwise it doesn't garbge collect
            self.hidden_state = hidden_state[-self.max_hidden_states:]
            self.z = z.data
            self.z_next = z_next.data

            latest_hidden = hidden_state[-1].squeeze(0) # squeeze so we can concat
        obs = torch.cat([z, latest_hidden], -1) 
        obs.requires_grad = True
        
        # Predict next action and value
        phi = self.network.phi_body(obs)
        phi_a = self.network.actor_body(phi)
        phi_v = self.network.critic_body(phi)
        prob = F.softmax(self.network.fc_action(phi_a), dim=-1)
        v = self.network.fc_critic(phi_v)
        dist = torch.distributions.Categorical(probs=prob)
        if action is None:
            action = dist.sample()
            self.render()
        log_prob = dist.log_prob(action).unsqueeze(-1)
        
        return action, log_prob, dist.entropy().unsqueeze(-1), v
    
    def render(self, mode='world_model', close=False):
        env=self
        if close:
            for viewer in [self.viewer_z, self.viewer_z_next, self.viewer_img_z, self.viewer_img_z_next]:
                viewer.close()
            if self.viewer:
                self.viewer.close()
            return
        if mode == 'world_model':
            # Render while showing latent vectors and decoded latent vectors
            if self.viewer is None:
                from gym.envs.classic_control.rendering import SimpleImageViewer
                import pyglet
                margin_vert = 60  # for window border
                # TODO add margin_horiz
                mult = 3

                self.viewer = SimpleImageViewer()
                self.viewer.window = pyglet.window.Window(width=160 * mult, height=128 * mult, vsync=False, resizable=True, caption='Game output')
                self.viewer.window.set_location(0 * mult, 0 * mult)

                self.viewer_img_z = SimpleImageViewer()
                self.viewer_img_z.window = pyglet.window.Window(width=128 * mult, height=128 * mult, vsync=False, resizable=True, caption='Decoded image')
                self.viewer_img_z.window.set_location(160 * mult, 0 * mult)

                self.viewer_img_z_next = SimpleImageViewer()
                self.viewer_img_z_next.window = pyglet.window.Window(width=128 * mult, height=128 * mult,
                                                                     vsync=False, resizable=True, caption='Decoded predicted image')
                self.viewer_img_z_next.window.set_location((160 + 128) * mult, 0 * mult)

                self.viewer_z = SimpleImageViewer()
                self.viewer_z.window = pyglet.window.Window(width=128 * mult, height=128 * mult, vsync=False, resizable=True, caption='latent vector')
                self.viewer_z.window.set_location(160 * mult, 128 * mult + margin_vert)

                self.viewer_z_next = SimpleImageViewer()
                self.viewer_z_next.window = pyglet.window.Window(width=128 * mult, height=128 * mult, vsync=False,
                                                                 resizable=True, caption='latent predicted vector')
                self.viewer_z_next.window.set_location((160 + 128) * mult, 128 * mult + margin_vert)

            # Decode latent vector for display
            with torch.no_grad():

                # to pytorch
                zv = self.z
                zv_next = self.z_next
                if self.cuda:
                    zv = zv.cuda()
                    zv_next = zv_next.cuda()

                # Decode
                img_z = self.world_model.vae.decode(zv)
                img_z_next = self.world_model.vae.decode(zv_next)

                # to numpy images
                img_z = img_z.squeeze(0).transpose(0, 2)
                img_z = img_z.data.cpu().numpy()
                img_z = (img_z * 255.0).astype(np.uint8)
                img_z_next = img_z_next.squeeze(0).transpose(0, 2).clamp(0, 1)
                img_z_next = img_z_next.data.cpu().numpy()
                img_z_next = (img_z_next * 255.0).astype(np.uint8)

                z_uint8 = ((self.z[0].data.cpu().numpy() + 0.5) * 255).astype(np.uint8).reshape((16, 16))
                z_uint8 = skimage.color.gray2rgb(z_uint8)  # Make it rgb to avoid problems with pyglet
                z_uint8 = cv2.resize(z_uint8, dsize=(256, 256), interpolation=cv2.INTER_NEAREST)  # Resize manually to avoid interp of pixels

                z_next_uint8 = ((self.z_next[0].data.cpu().numpy() + 0.5) * 255).astype(np.uint8).reshape((16, 16))
                z_next_uint8 = skimage.color.gray2rgb(z_next_uint8)
                z_next_uint8 = cv2.resize(z_next_uint8, dsize=(256, 256), interpolation=cv2.INTER_NEAREST)

                # Display
                img = self.img[0]*255
                self.viewer.imshow(img.astype(np.uint8))
                self.viewer_img_z.imshow(img_z)
                self.viewer_img_z_next.imshow(img_z_next)
                self.viewer_z.imshow(z_uint8)
                self.viewer_z_next.imshow(z_next_uint8)
                return self.viewer.isopen


In [20]:
log_dir = log_dir='./outputs/{NAME}'.format(NAME=NAME)
z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size
print(log_dir)


def task_fn(log_dir):
    return SonicWorldModelDeepRL(
        env_fn=lambda: make_env(
            'sonic256', state='GreenHillZone', game='SonicTheHedgehog-Genesis'),
        max_steps=1000,
        log_dir=log_dir,
        verbose=verbose
    )


config = Config()

config.num_workers = 3
config.task_fn = lambda: ParallelizedTask(
    task_fn, config.num_workers, single_process=config.num_workers == 1)
config.optimizer_fn = lambda params: torch.optim.RMSprop(params, 3e-4)
config.network_fn = lambda state_dim, action_dim: CategoricalActorCriticNet(
    state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), gpu=0 if cuda else -1, world_model_fn=lambda: world_model,
    render=(config.num_workers==1 and verbose)
)
config.discount = 0.99
config.logger = get_logger(
    NAME,
    file_name='deep_rl_ppo.log',
    level=logging.INFO,
    log_dir='./outputs/{NAME}'.format(NAME=NAME), )
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.0001
config.gradient_clip = 0.2
config.rollout_length = 128
config.optimization_epochs = 10
config.num_mini_batches = 4
config.ppo_ratio_clip = 0.2
config.iteration_log_interval = 1
agent = PPOAgent(config)
if os.path.isfile(ppo_save_file):
    print('loading', ppo_save_file)
    agent.load(ppo_save_file)

./outputs/RNN_v3b_256im_512z_v6_greenfield
game: SonicTheHedgehog-Genesis state: GreenHillZone.Act3
loading ./outputs/models/PPO_greenfields_256z_v5_wd_in_net.pkl


In [21]:
try:
    run_iterations(agent, log_dir=log_dir)
except:
    if config.num_workers==1:
        agent.task.tasks[0].env.close()
    else:
        [t.close() for t in agent.task.tasks]
    print("saving", ppo_save_file)
    agent.save(ppo_save_file)
    raise

2018-05-25 18:15:31,753 - RNN_v3b_256im_512z_v6_greenfield - INFO: total steps 128, min/mean/max reward 0.0000/0.0000/0.0000 of 1
2018-05-25 18:15:31,754 - RNN_v3b_256im_512z_v6_greenfield - INFO: running min/mean/max reward 0.0000/0.0000/0.0000 of 1 19.8994 s/rollout
2018-05-25 18:15:50,486 - RNN_v3b_256im_512z_v6_greenfield - INFO: total steps 256, min/mean/max reward 0.0000/0.0000/0.0000 of 1
2018-05-25 18:15:50,487 - RNN_v3b_256im_512z_v6_greenfield - INFO: running min/mean/max reward 0.0000/0.0000/0.0000 of 1 18.7332 s/rollout


saving ./outputs/models/PPO_greenfields_256z_v5_wd_in_net.pkl


KeyboardInterrupt: 

In [ ]:
# TODO work out the in an out shape I should handle in the two cases: rollout and sampling

In [ ]:
%debug

So when sampling the shape might be:
- in 
    - sampled_states.shape torch.Size([96, 256, 256, 3])
    - sampled_actions.shape torch.Size([96])
- out:
    - ipdb> log_probs.shape torch.Size([96, 1])
    - entropy_loss.shapetorch.Size([96, 1])
    - values.shape torch.Size([96, 1])
    
but why is sampled_log_probs_old.shape torch.Size([96, 3, 1]). I have 3 env's at once, but should they be concatnated? I need to check the output shape during rollout

ipdb> sampled_log_probs_old.shape torch.Size([96, 3, 1]) (WRONG)
ipdb> sampled_states.shape torch.Size([96, 256, 256, 3])
ipdb> sampled_actions.shape torch.Size([96])
ipdb> sampled_returns.shape torch.Size([96, 1])
ipdb> sampled_advantages.shape torch.Size([96, 1])

In [ ]:
world_model.cuda()
list(world_model.parameters())[0].is_cuda


To monitor with tensorboard
```sh
cd ~/Documents/projects/retro_sonic_comp/world-models-pytorch/log 
tensorboard  --logdir .
#then open http://localhost:6006/#scalars
```
